In [1]:
import keyring
import pandas as pd
import datetime
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pymongo import MongoClient

In [2]:
# Get dates for query
today = datetime.datetime.today()
one_week_ago = today - datetime.timedelta(days=7)

In [3]:
# Create connection string
user = "dbadmin"
password = keyring.get_password('docdb_prod_dbadmin', 'dbadmin')
host = "production-documentdb.cluster-cb6kajicuplh.us-east-1.docdb.amazonaws.com"
port = "27017"
options = "tls=true&tlsAllowInvalidCertificates=true&replicaSet=rs0&readPreference=secondaryPreferred&retryWrites=false"
cxn_string_template= "mongodb://{}:{}@{}:{}/?{}"
cxn_string_prod = cxn_string_template.format(user, password, host, port, options)


# Get past week's new products

In [4]:
# Get past week's new products
client = MongoClient(cxn_string_prod)
db = client['product-prod']
results_ = list(db.PdwProductCore.find({'createTimestamp': {'$gte': one_week_ago}}, 
                                       ('productGeneral.cusip', 'productGeneral.isin')))
cusips = [i['productGeneral']['cusip'] for i in results_ if 'cusip' in i['productGeneral']]
isins = [i['productGeneral']['isin'] for i in results_ if 'isin' in i['productGeneral']]

# Get BMO products added within first 5 pages

In [5]:
# # Setup for bmo
# bmo_act_dict = {}
# chrome_path = r"/opt/homebrew/bin/chromedriver"
# driver = webdriver.Chrome(chrome_path)
# url = 'https://www.bmonotes.com/Type/PPNs#active'
# # Get first page
# page = driver.get(url)
# time.sleep(2)
# bmo_act_dict[0] = pd.read_html(driver.page_source)[1]
# # Get remaining 4 pages
# for num in range(2, 6):
#     driver.find_element_by_id("DataTables_Table_1_next").click()
#     time.sleep(2)
#     bmo_act_dict[num] = pd.read_html(driver.page_source)[1]
# driver.close()
# # Combine the dataframes
# bmo_active_products = pd.concat([bmo_act_dict[k] for k in bmo_act_dict.keys()], ignore_index=True)
# # Create pdw cusip for comparison
# bmo_active_products['pdwCusip'] = [
#     'CA' + i if len(i) == 7 else 'C' + i for i in bmo_active_products['JHN Code / Cusip']
# ]
# # Filter out any products already present in pdw
# new_bmo_active_products = bmo_active_products[~bmo_active_products['JHN Code / Cusip'].isin(cusips) | 
#                                               bmo_active_products['JHN Code / Cusip'].isin(isins)]

In [ ]:

https://www.investorsolutions.gbm.scotiabank.com/ppn-public/home.do
https://notes.cibc.com/#/previouslyIssued

# Testing NBCSS

In [37]:
# Setup for nbc_ss
nbc_ss_act_dict = {}
chrome_path = r"/opt/homebrew/bin/chromedriver"
op = webdriver.ChromeOptions()
op.add_argument('--headless')
driver = webdriver.Chrome(chrome_path, options=op)
url = 'https://www.nbcstructuredsolutions.ca/listeProduits.aspx?mode=previous'
# Get first page
page = driver.get(url)
time.sleep(2)
nbc_ss_act_dict[0] = pd.read_html(driver.page_source)[0]
nbc_ss_act_dict[0]['urls'] = [driver.find_element_by_xpath('//*[@id="ctl00_cphMain_lvProducts_ctrl{}_lnkProduit"]'.format(i)).get_attribute('href') for i in range(len(nbc_ss_act_dict[0]))]
# Get remaining 4 pages
for num in range(2, 6):
    driver.find_element_by_xpath('//*[@id="ctl00_cphMain_dpProductsHaut"]/a[{}]'.format(str(num))).click()
    time.sleep(2)
    nbc_ss_act_dict[num] = pd.read_html(driver.page_source)[0]
    nbc_ss_act_dict[num]['urls'] = [driver.find_element_by_xpath('//*[@id="ctl00_cphMain_lvProducts_ctrl{}_lnkProduit"]'.format(i)).get_attribute('href') for i in range(len(nbc_ss_act_dict[num]))]
driver.close()
# Combine the dataframes
nbc_ss_active_products = pd.concat([nbc_ss_act_dict[k] for k in nbc_ss_act_dict.keys()], ignore_index=True)


[]


In [39]:
nbc_ss_active_products

,Name,FundSERV,Type,Term,Price,Issue Date,Maturity Date,urls
0,"NBC Auto Callable Canadian Equity Flex GIC, se...",NBC28229,GIC,7 years,NaN,15-Aug-2022,15-Aug-2029,https://www.nbcstructuredsolutions.ca/detailPr...
1,NBC Auto Callable Contingent Income Note Secur...,NBC25842,NPPN,7 years,NaN,15-Aug-2022,15-Aug-2029,https://www.nbcstructuredsolutions.ca/detailPr...
2,NBC Auto Callable Contingent Income Note Secur...,NBC25841,NPPN,5 years,NaN,15-Aug-2022,16-Aug-2027,https://www.nbcstructuredsolutions.ca/detailPr...
3,NBC Auto Callable Contingent Income Note Secur...,NBC25840,NPPN,5 years,NaN,12-Aug-2022,12-Aug-2027,https://www.nbcstructuredsolutions.ca/detailPr...
4,NBC Auto Callable Note Securities (Maturity-Mo...,NBC24315,NPPN,5 years,NaN,12-Aug-2022,12-Aug-2027,https://www.nbcstructuredsolutions.ca/detailPr...
...,...,...,...,...,...,...,...,...
70,NBC Auto Callable Note Securities (Maturity-Mo...,NBC24302,NPPN,5 years,$102.75,13-Jul-2022,13-Jul-2027,https://www.nbcstructuredsolutions.ca/detailPr...
71,"NBC Canadian Equity Flex GIC, series 43F",NBC28207,GIC,4.5 years,$100.10,13-Jul-2022,13-Jan-2027,https://www.nbcstructuredsolutions.ca/detailPr...
72,NBC Auto Callable Contingent Income Note Secur...,NBC25804,NPPN,7 years,$100.84,13-Jul-2022,13-Jul-2029,https://www.nbcstructuredsolutions.ca/detailPr...
73,NBC Auto Callable Note Securities (Maturity-Mo...,NBC24300,NPPN,7 years,$101.12,12-Jul-2022,12-Jul-2029,https://www.nbcstructuredsolutions.ca/detailPr...


# Testing rbcnotes

In [ ]:
//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[3]/td[3]/a
//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[5]/td[3]/a

In [41]:
# Setup for nbc_ss
rbc_act_dict = {}
chrome_path = r"/opt/homebrew/bin/chromedriver"
op = webdriver.ChromeOptions()
op.add_argument('--headless')
driver = webdriver.Chrome(chrome_path, options=op)
url = 'https://www.rbcnotes.com/Products'
# Get first page
page = driver.get(url)
time.sleep(2)
rbc_act_dict[0] = pd.read_html(driver.page_source)[0]
rbc_act_dict[0]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[1]/td[3]/a'.format(i)).get_attribute('href') 
                                for i in range(1, len(rbc_act_dict[0])-1, 2)]
# Get remaining 4 pages
for num in range(2, 6):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[4]/a[3]').click()
    time.sleep(2)
    rbc_act_dict[num] = pd.read_html(driver.page_source)[0]
    rbc_act_dict[num]['urls'] = [driver.find_element_by_xpath('//*[@id="productGrid"]/div[2]/div[3]/table/tbody/tr[{}]/td[3]/a'.format(i)).get_attribute('href') 
                                    for i in range(1, len(rbc_act_dict[0])-1, 2)]
driver.close()
# Combine the dataframes
rbc_active_products = pd.concat([rbc_act_dict[k] for k in rbc_act_dict.keys()], ignore_index=True)


ElementClickInterceptedException: Message: element click intercepted: Element <span class="k-icon k-i-arrow-e"></span> is not clickable at point (188, 592). Other element would receive the click: <div role="dialog" aria-live="polite" aria-label="cookieconsent" aria-describedby="cookieconsent:desc" class="cc-window cc-banner cc-type-opt-out cc-theme-block cc-bottom cc-color-override--364807516 " style="">...</div>
  (Session info: headless chrome=104.0.5112.79)
